In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import pandas as pd
import multiprocessing
from scipy import signal
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from model_src.DilatedResNet import DilatedResNet
from model_src.BianResnet import BianResNet
from model_src.LSTM import VanillaLSTM, CNNLSTM, BiLSTM, BiLSTMAttn
from model_src.RespNet import RespNet

physical_devices = tf.config.list_physical_devices('GPU')
print(f'Is GPU Avaliable: {physical_devices[0]}')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
DATA_PATH = '/root/Workspace/DataLake/stMary'
DATA_SAVE_PATH = '/root/Workspace/Project-RRpo-2ndStudy/dataset' 

Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU Avaliable: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [2]:
EPOCHS = 1000
BATCH_SIZE = 256
LR = 0.001
callbacks = [
    EarlyStopping(monitor='val_loss', patience=33),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)
]

In [3]:
def gen_tfdataset(dataset, batchsize):
    X = []; y = []
    for pleth, resp in dataset:
        X.append(pleth.astype(np.float32))
        y.append(resp)

    X = np.array(X); y = np.array(y).reshape(-1,1)
    scaler = MinMaxScaler()
    scaled_X = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in X])
    print(f'Overall: {scaled_X.shape}, {y.shape}')
    return tf.data.Dataset.from_tensor_slices((scaled_X, y)).batch(batchsize)


def cross_validation(model, model_name, dataset, dataset_name, n_splits=5, batch_size=256, lr=0.001):
    # kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    kf = KFold(n_splits=n_splits)
    train_losses = []; val_losses = []
    subject_id = np.array([subject[0] for subject in dataset])
    count = 1
    
    for train_idx, val_idx in kf.split(subject_id):
        train_dataset = []; val_dataset = []
        # print(subject_id[train_idx])
        # print(subject_id[val_idx])
        for id, samples in dataset:
            # if id not in ['bidmc_13', 'bidmc_19']:
                if id in subject_id[train_idx]: train_dataset.extend(samples)
                else: val_dataset.extend(samples)
        
        train_dataset = np.array(train_dataset); val_dataset = np.array(val_dataset)

        train_tf = gen_tfdataset(train_dataset, batch_size)
        val_tf = gen_tfdataset(val_dataset, batch_size)

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
            loss=keras.losses.MeanAbsoluteError(),
            metrics=keras.metrics.MeanAbsoluteError()
        )  

        callbacks.append(ModelCheckpoint(
            filepath=f'../models/230921/{dataset_name}/{model_name}/{model_name}-{dataset_name}-KF{count}/ckpt', 
            monitor='val_loss',
            verbose=0,
            save_best_only=True,
            save_weights_only=True))

        history = model.fit(
            train_tf,
            epochs=EPOCHS,
            callbacks=callbacks,
            validation_data=val_tf
        )

        callbacks.pop()

        min_val_loss_idx = np.argmin(history.history['val_loss'])
        train_losses.append(history.history['loss'][min_val_loss_idx])
        val_losses.append(history.history['val_loss'][min_val_loss_idx])
        count = count + 1
    
    print(f'TRAIN: {np.mean(train_losses)} ± {np.std(train_losses)}')
    print(f'VAL: {np.mean(val_losses)} ± {np.std(val_losses)}')
    return train_losses, val_losses

### Experiments
성모병원 데이터셋 `stmary-trainval_dataset.npy`를 이용해서 7개의 모델을 학습시킨다. 

해당 학습 모델들은 다음과 같이 저장시킨다: `stmary-<Model name>`

이외의 학습 파라메터는 아래와 같다:
- Epochs: `1000`

- Batch size: `256`

- Learning rate: `0.001`

- callbacks: `EarlyStopping(monitor='val_loss', patience=33),ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)`

또한 7개의 모델 각각에 대한 하이퍼 파라메터 조정은 아래와 같이 이루어졌다:
- Bian model: 논문(`10.1109/EMBC44109.2020.9176231`)의 파라메터를 따랐다.

- Unet model: 논문(`10.1109/embc.2019.8856301`)의 파라메터를 따랐다.

- LSTM family model: 논문(`10.1016/j.compbiomed.2022.105338`)의 파라메터를 따랐다.

- Proposed model: Bayesian optimization을 적용하였다.
    + `num_of_blocks`: RespBlk의 블록 개수 [1 ~ 5]

    + `kernel_size`: RespBlk의 kernel 크기 [2 ~ 5]

    + `dilation_rate`: RespBlk 내 Convolution layer들의 Dilation 정도 [0 ~ 5]

    + `dwn_kernel_size`: Downsample을 위한 1Dconvolution의 kernel_size [2 ~ 4]

    + `filters`: 필터 개수, 블록의 반복마다 2의 제곱배 되어 증가한다.

    + `strides`: Average pooling의 stride 크기 [2 ~ 4]

    + `units`: 첫번째 Dense layer의 unit 수 [20 ~ 100]

- 학습할 모델:
    모델을 학습할 때는 `LOSO method`를 따르며, `5-Fold CV`를 시행한다.
    + 1. StMary-based Models: `stmary-trainval_dataset.npy`으로 학습 및 조정한다.

    + 2. BIDMC-based Models: `bidmc-trainval_dataset.npy`으로 학습 및 조정한다.

- Experiment 1:
    + 1번 모델을 `stmary-test_dataset.npy`로 최종 검증한다.

- Experiment 2:
    + 1번 모델을 `stmary-test_dataset.npy`로 호흡 속도에 따라 따로 검증한다.

- Experiment 3:
    + 1번 모델을 `bidmc-preprocessed.npy`와 `capno-preprocessed.npy`, 그리고 그 각각의 호흡 속도에 따라 따로 검증한다.

- Experiment 4:
    + 2번 모델을 `stmary-preprocessed.npy`와 `capno-preprocessed.npy`, 그리고 그 각각의 호흡 속도에 따라 따로 검증한다.

## Modeling with StMary

In [4]:
stmary = np.load(f'{DATA_SAVE_PATH}/230920/stmary-trainval_dataset.npy', allow_pickle=True)
print(stmary.shape)

(85, 2)


In [ ]:
# models = [BianResNet(), RespNet(), VanillaLSTM(), CNNLSTM(), BiLSTM(), BiLSTMAttn()]
# model_names = ['Bian', 'RespNet', 'LSTM', 'CNNLSTM', 'BiLSTM', 'BiLSTMAttn']
models = [BiLSTM(), BiLSTMAttn()]
model_names = ['BiLSTM', 'BiLSTMAttn']

In [ ]:
results = np.array([(model_names[i], cross_validation(model, model_name=model_names[i], dataset=stmary, dataset_name='stmary', n_splits=5, batch_size=BATCH_SIZE, lr=LR)) for i, model in enumerate(models)])

## Modeling with BIDMC

In [5]:
bidmc = np.load(f'{DATA_SAVE_PATH}/230921/bidmc-trainval_dataset.npy', allow_pickle=True)
print(bidmc.shape)

(43, 2)


In [6]:
# models = [BianResNet(), RespNet(), VanillaLSTM(), CNNLSTM(), BiLSTM(), BiLSTMAttn()]
# model_names = ['Bian', 'RespNet', 'LSTM', 'CNNLSTM', 'BiLSTM', 'BiLSTMAttn']
models = [RespNet(), VanillaLSTM(), CNNLSTM(), BiLSTM(), BiLSTMAttn()]
model_names = ['RespNet', 'LSTM', 'CNNLSTM', 'BiLSTM', 'BiLSTMAttn']

In [15]:
results = np.array([(model_names[i], cross_validation(model, model_name=model_names[i], dataset=bidmc, dataset_name='bidmc', n_splits=5, batch_size=BATCH_SIZE, lr=LR)) for i, model in enumerate(models)])

Overall: (14314, 1800, 1), (14314, 1)
Overall: (3789, 1800, 1), (3789, 1)
Epoch 1/1000
56/56 [==============================] - 84s 718ms/step - loss: 16.9770 - mean_absolute_error: 16.9770 - val_loss: 16.7179 - val_mean_absolute_error: 16.7179 - lr: 0.0010
Epoch 2/1000
56/56 [==============================] - 35s 620ms/step - loss: 9.1057 - mean_absolute_error: 9.1057 - val_loss: 2.2359 - val_mean_absolute_error: 2.2359 - lr: 0.0010
Epoch 3/1000
56/56 [==============================] - 34s 617ms/step - loss: 2.2488 - mean_absolute_error: 2.2488 - val_loss: 1.6418 - val_mean_absolute_error: 1.6418 - lr: 0.0010
Epoch 4/1000
56/56 [==============================] - 34s 609ms/step - loss: 2.1402 - mean_absolute_error: 2.1402 - val_loss: 1.7563 - val_mean_absolute_error: 1.7563 - lr: 0.0010
Epoch 5/1000
56/56 [==============================] - 34s 611ms/step - loss: 2.0354 - mean_absolute_error: 2.0354 - val_loss: 2.0440 - val_mean_absolute_error: 2.0440 - lr: 0.0010
Epoch 6/1000
56/56 [==

In [16]:
results

array([['RespNet',
        ([2.2487967014312744, 1.5657647848129272, 0.9786404967308044, 1.2727805376052856, 1.090259313583374], [1.6417512893676758, 1.555878758430481, 2.465959072113037, 1.2691096067428589, 1.7008860111236572])],
       ['LSTM',
        ([3.0102298259735107, 1.8266541957855225, 1.61507248878479, 2.255537748336792, 1.7941203117370605], [1.5839589834213257, 1.5861732959747314, 2.385347604751587, 1.2881839275360107, 1.6983428001403809])],
       ['CNNLSTM',
        ([3.5285983085632324, 2.063640594482422, 1.6176753044128418, 2.30708646774292, 1.7954273223876953], [1.612129807472229, 1.5933068990707397, 2.3923544883728027, 1.2473341226577759, 1.707406997680664])],
       ['BiLSTM',
        ([3.6196131706237793, 1.8414870500564575, 1.6202541589736938, 2.2670488357543945, 1.7965338230133057], [1.5180238485336304, 1.5879298448562622, 2.393948554992676, 1.2257028818130493, 1.703553318977356])],
       ['BiLSTMAttn',
        ([3.1541190147399902, 2.3293635845184326, 1.57219827

### Check Best

In [8]:
def check_result(model_path, models, model_names, dataset, dataset_name, n_splits):
    # validation
    kf = KFold(n_splits=n_splits)
    train_losses = []; val_losses = []
    subject_id = np.array([subject[0] for subject in dataset])

    for model_name, model in zip(model_names, models):
        train_loss = []; val_loss = []
        count = 0
        loaded_files = sorted(os.listdir(f'{model_path}/{dataset_name}/{model_name}'))
        for train_idx, val_idx in kf.split(subject_id):
            load_model = model()
            load_model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=keras.losses.MeanAbsoluteError(),
                metrics=keras.metrics.MeanAbsoluteError()
            )
            load_model.load_weights(f"{model_path}/{dataset_name}/{model_name}/{loaded_files[count]}/ckpt")
            train_dataset = []; val_dataset = []
            
            for id, samples in dataset:
                if id in subject_id[train_idx]: train_dataset.extend(samples)
                else: val_dataset.extend(samples)
            
            train_dataset = np.array(train_dataset); val_dataset = np.array(val_dataset)

            train_tf = gen_tfdataset(train_dataset, 256)
            val_tf = gen_tfdataset(val_dataset, 256)

            train_loss.append(load_model.evaluate(train_tf)[0])
            val_loss.append(load_model.evaluate(val_tf)[0])
            count += 1
        train_losses.append(train_loss)
        val_losses.append(val_loss)
    return np.array(train_losses), np.array(val_losses)

In [9]:
train_losses, val_losses = check_result(
    model_path='../models/230921/',
    models=[BianResNet],
    model_names=['Bian'],
    dataset=bidmc,
    dataset_name='bidmc',
    n_splits=5
)

Overall: (14314, 1800, 1), (14314, 1)
Overall: (3789, 1800, 1), (3789, 1)
15/15 [==============================] - 0s 24ms/step - loss: 0.7425 - mean_absolute_error: 0.7425
Overall: (14314, 1800, 1), (14314, 1)
Overall: (3789, 1800, 1), (3789, 1)
15/15 [==============================] - 0s 10ms/step - loss: 1.0327 - mean_absolute_error: 1.0327
Overall: (14314, 1800, 1), (14314, 1)
Overall: (3789, 1800, 1), (3789, 1)
15/15 [==============================] - 0s 11ms/step - loss: 2.0989 - mean_absolute_error: 2.0989
Overall: (14735, 1800, 1), (14735, 1)
Overall: (3368, 1800, 1), (3368, 1)
14/14 [==============================] - 0s 19ms/step - loss: 0.9117 - mean_absolute_error: 0.9117
Overall: (14735, 1800, 1), (14735, 1)
Overall: (3368, 1800, 1), (3368, 1)
14/14 [==============================] - 0s 11ms/step - loss: 0.9530 - mean_absolute_error: 0.9530


In [10]:
print(np.array(train_losses), np.mean(np.array(train_losses), axis=1), np.std(np.array(train_losses), axis=1))
print(np.array(val_losses), np.mean(np.array(val_losses), axis=1), np.std(np.array(val_losses), axis=1))

[[1.21145749 1.38949728 1.00850999 1.45145667 1.04009724]] [1.22020373] [0.17858741]
[[0.74252307 1.03270996 2.09892273 0.9116596  0.95303637]] [1.14777035] [0.48493808]


In [ ]:
# # validation
# kf = KFold(n_splits=5)
# train_losses = []; val_losses = []
# count = 0

# models = [BianResNet, RespNet, VanillaLSTM, CNNLSTM, BiLSTM, BiLSTMAttn]
# model_names = ['Bian', 'RespNet', 'LSTM', 'CNNLSTM', 'BiLSTM', 'BiLSTMAttn']
# dataset_name = 'bidmc'
# dataset = bidmc
# subject_id = np.array([subject[0] for subject in dataset])

# for model_name, model in zip(model_names, models):
#     val_loss = []
#     count = 0
#     loaded_files = sorted(os.listdir(f'../models/230921/{dataset_name}/{model_name}'))
#     for train_idx, val_idx in kf.split(subject_id):
#         load_model = model()
#         load_model.compile(
#             optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#             loss=keras.losses.MeanAbsoluteError(),
#             metrics=keras.metrics.MeanAbsoluteError()
#         )
#         load_model.load_weights(f"../models/230921/{dataset_name}/{model_name}/{loaded_files[count]}/ckpt")
#         train_dataset = []; val_dataset = []
        
#         for id, samples in dataset:
#             if id in subject_id[train_idx]: train_dataset.extend(samples)
#             else: val_dataset.extend(samples)
        
#         train_dataset = np.array(train_dataset); val_dataset = np.array(val_dataset)

#         train_tf = gen_tfdataset(train_dataset, 256)
#         val_tf = gen_tfdataset(val_dataset, 256)

#         val_loss.append(load_model.evaluate(val_tf)[0])
#         count += 1
#     val_losses.append(val_loss)

### Test

In [11]:
def get_resp_class(dataset):
    rapid_rr = []; normal_rr = []; slow_rr = []
    for id, samples in dataset:
        for sample in samples:
            if sample[1] < 12: slow_rr.append(sample)
            elif sample[1] > 20: rapid_rr.append(sample)
            else: normal_rr.append(sample)
    
    rapid_rr = np.array(rapid_rr)
    normal_rr = np.array(normal_rr)
    slow_rr = np.array(slow_rr)
    return rapid_rr, normal_rr, slow_rr


def get_trainable_X(dataset):
    scaler = MinMaxScaler()
    return np.expand_dims(scaler.fit_transform(np.vstack(dataset[:, 0])),axis=-1)


def get_predict_performance(model, X, y):
    y_pred = model.predict(X)
    return y - y_pred


def test_model(model_path='../models/230921', model=None, model_name=None, data_path='../dataset/test_dataset'):
    if model_name == 'stmary': stmary_test = np.load(f'{data_path}/stmary-test_dataset.npy', allow_pickle=True)
    else: stmary_test = np.load(f'{data_path}/stmary-preprocessed.npy', allow_pickle=True)
    stmary_rapid, stmary_normal, stmary_slow = get_resp_class(stmary_test)
    stmary_test = np.concatenate([stmary_rapid, stmary_normal, stmary_slow],axis=0)
    # stmary_tf_rapid, stmary_tf_normal, stmary_tf_slow, stmary_tf_test = gen_tfdataset(stmary_rapid, 256), gen_tfdataset(stmary_normal, 256), gen_tfdataset(stmary_slow, 256), gen_tfdataset(stmary_test, 256)
    
    X_stmary_np_rapid, y_stmary_np_rapid = get_trainable_X(stmary_rapid), stmary_rapid[:, 1].reshape(-1,1)
    X_stmary_np_normal, y_stmary_np_normal = get_trainable_X(stmary_normal), stmary_normal[:, 1].reshape(-1,1)
    X_stmary_np_slow, y_stmary_np_slow = get_trainable_X(stmary_slow), stmary_slow[:, 1].reshape(-1,1)
    X_stmary_np_test, y_stmary_np_test = get_trainable_X(stmary_test), stmary_test[:, 1].reshape(-1,1)

    if model_name == 'bidmc': bidmc_test = np.load(f'{data_path}/bidmc-test_dataset.npy', allow_pickle=True)
    else: bidmc_test = np.load(f'{data_path}/bidmc-preprocessed.npy', allow_pickle=True)
    bidmc_rapid, bidmc_normal, bidmc_slow = get_resp_class(bidmc_test)
    bidmc_test = np.concatenate([bidmc_rapid, bidmc_normal, bidmc_slow],axis=0)
    # bidmc_tf_rapid, bidmc_tf_normal, bidmc_tf_slow, bidmc_tf_test = gen_tfdataset(bidmc_rapid, 256), gen_tfdataset(bidmc_normal, 256), gen_tfdataset(bidmc_slow, 256), gen_tfdataset(bidmc_test, 256)

    X_bidmc_np_rapid, y_bidmc_np_rapid = get_trainable_X(bidmc_rapid), bidmc_rapid[:, 1].reshape(-1,1)
    X_bidmc_np_normal, y_bidmc_np_normal = get_trainable_X(bidmc_normal), bidmc_normal[:, 1].reshape(-1,1)
    X_bidmc_np_slow, y_bidmc_np_slow = get_trainable_X(bidmc_slow), bidmc_slow[:, 1].reshape(-1,1)
    X_bidmc_np_test, y_bidmc_np_test = get_trainable_X(bidmc_test), bidmc_test[:, 1].reshape(-1,1)

    capno_test = np.load(f'{data_path}/capno-preprocessed.npy', allow_pickle=True)
    capno_rapid, capno_normal, capno_slow = get_resp_class(capno_test)
    capno_test = np.concatenate([capno_rapid, capno_normal, capno_slow],axis=0)
    # capno_tf_rapid, capno_tf_normal, capno_tf_slow, capno_tf_test = gen_tfdataset(capno_rapid, 256), gen_tfdataset(capno_normal, 256), gen_tfdataset(capno_slow, 256), gen_tfdataset(capno_test, 256)

    X_capno_np_rapid, y_capno_np_rapid = get_trainable_X(capno_rapid), capno_rapid[:, 1].reshape(-1,1)
    X_capno_np_normal, y_capno_np_normal = get_trainable_X(capno_normal), capno_normal[:, 1].reshape(-1,1)
    X_capno_np_slow, y_capno_np_slow = get_trainable_X(capno_slow), capno_slow[:, 1].reshape(-1,1)
    X_capno_np_test, y_capno_np_test = get_trainable_X(capno_test), capno_test[:, 1].reshape(-1,1)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=keras.losses.MeanAbsoluteError(),
        metrics=keras.metrics.MeanAbsoluteError()
    )

    err_stmary_slow = get_predict_performance(model, X_stmary_np_slow, y_stmary_np_slow)
    err_stmary_normal = get_predict_performance(model, X_stmary_np_normal, y_stmary_np_normal)
    err_stmary_rapid = get_predict_performance(model, X_stmary_np_rapid, y_stmary_np_rapid)
    err_stmary_test = get_predict_performance(model, X_stmary_np_test, y_stmary_np_test)

    err_bidmc_slow = get_predict_performance(model, X_bidmc_np_slow, y_bidmc_np_slow)
    err_bidmc_normal = get_predict_performance(model, X_bidmc_np_normal, y_bidmc_np_normal)
    err_bidmc_rapid = get_predict_performance(model, X_bidmc_np_rapid, y_bidmc_np_rapid)
    err_bidmc_test = get_predict_performance(model, X_bidmc_np_test, y_bidmc_np_test)

    err_capno_slow = get_predict_performance(model, X_capno_np_slow, y_capno_np_slow)
    err_capno_normal = get_predict_performance(model, X_capno_np_normal, y_capno_np_normal)
    err_capno_rapid = get_predict_performance(model, X_capno_np_rapid, y_capno_np_rapid)
    err_capno_test = get_predict_performance(model, X_capno_np_test, y_capno_np_test)

    return {
        'stmary': [f'{np.mean(np.abs(err_stmary_slow))} ± {np.std(np.abs(err_stmary_slow))}',
                   f'{np.mean(np.abs(err_stmary_normal))} ± {np.std(np.abs(err_stmary_normal))}',
                   f'{np.mean(np.abs(err_stmary_rapid))} ± {np.std(np.abs(err_stmary_rapid))}',
                   f'{np.mean(np.abs(err_stmary_test))} ± {np.std(np.abs(err_stmary_test))}'],
        'bidmc': [f'{np.mean(np.abs(err_bidmc_slow))} ± {np.std(np.abs(err_bidmc_slow))}',
                  f'{np.mean(np.abs(err_bidmc_normal))} ± {np.std(np.abs(err_bidmc_normal))}',
                  f'{np.mean(np.abs(err_bidmc_rapid))} ± {np.std(np.abs(err_bidmc_rapid))}',
                  f'{np.mean(np.abs(err_bidmc_test))} ± {np.std(np.abs(err_bidmc_test))}'],
        'capno': [f'{np.mean(np.abs(err_capno_slow))} ± {np.std(np.abs(err_capno_slow))}',
                  f'{np.mean(np.abs(err_capno_normal))} ± {np.std(np.abs(err_capno_normal))}',
                  f'{np.mean(np.abs(err_capno_rapid))} ± {np.std(np.abs(err_capno_rapid))}',
                  f'{np.mean(np.abs(err_capno_test))} ± {np.std(np.abs(err_capno_test))}']                   
    }

    # return {
    #     'stmary': [model.evaluate(stmary_tf_slow)[0], model.evaluate(stmary_tf_normal)[0], model.evaluate(stmary_tf_rapid)[0], model.evaluate(stmary_tf_test)[0]],
    #     'bidmc': [model.evaluate(bidmc_tf_slow)[0], model.evaluate(bidmc_tf_normal)[0], model.evaluate(bidmc_tf_rapid)[0], model.evaluate(bidmc_tf_test)[0]],
    #     'capno': [model.evaluate(capno_tf_slow)[0], model.evaluate(capno_tf_normal)[0], model.evaluate(capno_tf_rapid)[0], model.evaluate(capno_tf_test)[0]]
    # }

In [12]:
loaded_model = BianResNet()
loaded_model.load_weights('../models/230921/bidmc/Bian/Bian-bidmc-KF1/ckpt')

In [13]:
test_result = test_model(model=loaded_model, model_name='bidmc')

172/172 [==============================] - 2s 9ms/step


In [14]:
test_result

{'stmary': ['7.7819213877963485 ± 1.5702955606375066',
  '2.1857326836817332 ± 1.60157975886301',
  '6.993027425487997 ± 4.0840639278767625',
  '5.168321436141706 ± 4.059956563722019'],
 'bidmc': ['6.301911354064941 ± 0.9845117996589524',
  '2.012267608942554 ± 1.4100085144343018',
  '3.092131222090475 ± 1.1425412948944311',
  '2.7833847166280807 ± 1.7148598902553314'],
 'capno': ['5.942704773363088 ± 1.3554517776478783',
  '1.7863607451203583 ± 1.0677781389859766',
  '7.678556205323024 ± 2.30276410492147',
  '4.443339580812599 ± 2.9707212004246357']}